Week 3 Assignment/Part 1.

In [1]:
# !pip3 install BeautifulSoup4
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r  = requests.get(url)
data = r.text
soup = BeautifulSoup(data)
# because this wiki page contains only 1 table, let's directly take the table out
table = soup.find('table', attrs={'class': 'wikitable sortable'})

Add to a dataframe.

In [3]:
neighborhood = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])

for tr in table.find_all('tr')[1:]:
    tds = tr.find_all('td')
    row = [tr.text.strip() for tr in tds if tr.text.strip()]
    neighborhood = neighborhood.append({'PostalCode': row[0],
                                        'Borough': row[1],
                                        'Neighborhood': row[2]}, ignore_index = True)
    
neighborhood.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Exclude rows that have unassigned borough.

In [4]:
neighborhood_excl = neighborhood[neighborhood['Borough'] != 'Not assigned']
neighborhood_excl.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Combine rows with the same postal code.

In [5]:
neighborhood_df = neighborhood_excl.groupby(['PostalCode', 'Borough']).agg(', '.join)
neighborhood_df.reset_index(inplace = True)
neighborhood_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Fix Not assigned neighborhoods.

In [8]:
unassigned_neighs = neighborhood_df['Neighborhood'] == 'Not assigned'
neighborhood_df.loc [unassigned_neighs,'Neighborhood'] = neighborhood_df.loc[unassigned_neighs,'Borough']
neighborhood_df.loc [unassigned_neighs]

,PostalCode,Borough,Neighborhood


Data size.

In [9]:
print('The final dataframe include', neighborhood_df.shape[0], 'rows and', neighborhood_df.shape[1], 'columns')
neighborhood_df

The final dataframe include 103 rows and 3 columns


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [10]:
neighborhood_df.to_pickle('3.1.pkl')